In [1]:
# 400 meter coordinate accuracy
# 1923 earliest year
import pandas as pd
import geopandas as gpd
from dask import dataframe as dd
import dask_geopandas

pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)


In [2]:
# read in cleaned parquet
df_path = input("Enter the file path: ")

df = dd.read_parquet(df_path)


In [3]:
# read in eco_map.geojson
map_path = input("Enter the file path: ")

map = gpd.read_file(map_path)


In [ ]:
df


In [ ]:
map.head()


In [4]:
map.crs


<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
# convert to dask_geopandas df
df_gpd = dask_geopandas.from_dask_dataframe(df)


In [6]:
del df, df_path

In [7]:
df_gpd = df_gpd.set_geometry(
    dask_geopandas.points_from_xy(
        df_gpd, "decimalLongitude", "decimalLatitude")
)


In [8]:
df_gpd = df_gpd.drop(["decimalLatitude", "decimalLongitude"], axis=1)


In [9]:
df_gpd = df_gpd.set_crs("EPSG:4326")

# if get stuck might be useful at some point
# df_gpd.drop(df_gpd[df_gpd["geometry"].is_empty].index, inplace=True)

# df_gpd["geometry"] = df_gpd["geometry"].apply(wkt.loads)
# df_gpd = gpd.GeoDataFrame(df_gpd, crs="EPSG:4326")

In [10]:
df_gpd.crs


<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [12]:
df_gpd = df_gpd.compute()

# df_gpd.head()


In [13]:
df_gpd = df_gpd.sjoin(map, predicate="within")


In [14]:
df_gpd


,kingdom,phylum,class,order,family,genus,species,license,rightsHolder,geometry,index_right,unique_id,name,TYPE
0,Animalia,Chordata,Actinopteri,Perciformes,Trachinidae,Echiichthys,Echiichthys vipera,None,None,POINT (4.36620 52.68900),13706,846,North Sea,MEOW
1,Animalia,Chordata,Actinopteri,Gadiformes,Merlucciidae,Merluccius,Merluccius merluccius,None,None,POINT (-0.56170 58.16970),13706,846,North Sea,MEOW
2,Animalia,Chordata,Actinopteri,Perciformes,Trachinidae,Echiichthys,Echiichthys vipera,None,None,POINT (2.58400 54.78870),13706,846,North Sea,MEOW
3,Animalia,Chordata,Actinopteri,Perciformes,Trachinidae,Echiichthys,Echiichthys vipera,None,None,POINT (2.58400 54.78870),13706,846,North Sea,MEOW
4,Animalia,Chordata,Actinopteri,Gadiformes,Merlucciidae,Merluccius,Merluccius merluccius,None,None,POINT (1.35930 58.85780),13706,846,North Sea,MEOW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10692867,Animalia,Chordata,Mammalia,Carnivora,Phocidae,Phoca,Phoca vitulina,http://creativecommons.org/licenses/by-nc/4.0/,None,POINT (-68.64867 44.04926),7210,321,New England-Acadian forests,TEOW
10695071,Animalia,Chordata,Mammalia,Carnivora,Phocidae,Phoca,Phoca vitulina,http://creativecommons.org/licenses/by-nc/4.0/,None,POINT (-68.64867 44.04926),7210,321,New England-Acadian forests,TEOW
10738560,Animalia,Chordata,Mammalia,Carnivora,Phocidae,Phoca,Phoca vitulina,http://creativecommons.org/licenses/by-nc/4.0/,None,POINT (-68.65000 44.03300),7210,321,New England-Acadian forests,TEOW
10738562,Animalia,Chordata,Mammalia,Carnivora,Phocidae,Phoca,Phoca vitulina,http://creativecommons.org/licenses/by-nc/4.0/,None,POINT (-70.01700 41.35000),4774,332,Atlantic coastal pine barrens,TEOW


In [15]:
del map, map_path

In [16]:
df_gpd = df_gpd.drop(["index_right", "geometry", "name", "TYPE"], axis=1)

In [17]:
df_gpd


,kingdom,phylum,class,order,family,genus,species,license,rightsHolder,unique_id
0,Animalia,Chordata,Actinopteri,Perciformes,Trachinidae,Echiichthys,Echiichthys vipera,None,None,846
1,Animalia,Chordata,Actinopteri,Gadiformes,Merlucciidae,Merluccius,Merluccius merluccius,None,None,846
2,Animalia,Chordata,Actinopteri,Perciformes,Trachinidae,Echiichthys,Echiichthys vipera,None,None,846
3,Animalia,Chordata,Actinopteri,Perciformes,Trachinidae,Echiichthys,Echiichthys vipera,None,None,846
4,Animalia,Chordata,Actinopteri,Gadiformes,Merlucciidae,Merluccius,Merluccius merluccius,None,None,846
...,...,...,...,...,...,...,...,...,...,...
10692867,Animalia,Chordata,Mammalia,Carnivora,Phocidae,Phoca,Phoca vitulina,http://creativecommons.org/licenses/by-nc/4.0/,None,321
10695071,Animalia,Chordata,Mammalia,Carnivora,Phocidae,Phoca,Phoca vitulina,http://creativecommons.org/licenses/by-nc/4.0/,None,321
10738560,Animalia,Chordata,Mammalia,Carnivora,Phocidae,Phoca,Phoca vitulina,http://creativecommons.org/licenses/by-nc/4.0/,None,321
10738562,Animalia,Chordata,Mammalia,Carnivora,Phocidae,Phoca,Phoca vitulina,http://creativecommons.org/licenses/by-nc/4.0/,None,332


In [18]:
# combine license and rightsHolder into dict and merge in single column
rights = ["license", "rightsHolder"]
df_gpd["rights"] = df_gpd[rights].to_dict(orient="records")
df_gpd = df_gpd.drop(["license", "rightsHolder"], axis=1)


In [19]:
df_gpd


,kingdom,phylum,class,order,family,genus,species,unique_id,rights
0,Animalia,Chordata,Actinopteri,Perciformes,Trachinidae,Echiichthys,Echiichthys vipera,846,"{'license': None, 'rightsHolder': None}"
1,Animalia,Chordata,Actinopteri,Gadiformes,Merlucciidae,Merluccius,Merluccius merluccius,846,"{'license': None, 'rightsHolder': None}"
2,Animalia,Chordata,Actinopteri,Perciformes,Trachinidae,Echiichthys,Echiichthys vipera,846,"{'license': None, 'rightsHolder': None}"
3,Animalia,Chordata,Actinopteri,Perciformes,Trachinidae,Echiichthys,Echiichthys vipera,846,"{'license': None, 'rightsHolder': None}"
4,Animalia,Chordata,Actinopteri,Gadiformes,Merlucciidae,Merluccius,Merluccius merluccius,846,"{'license': None, 'rightsHolder': None}"
...,...,...,...,...,...,...,...,...,...
10692867,Animalia,Chordata,Mammalia,Carnivora,Phocidae,Phoca,Phoca vitulina,321,{'license': 'http://creativecommons.org/licens...
10695071,Animalia,Chordata,Mammalia,Carnivora,Phocidae,Phoca,Phoca vitulina,321,{'license': 'http://creativecommons.org/licens...
10738560,Animalia,Chordata,Mammalia,Carnivora,Phocidae,Phoca,Phoca vitulina,321,{'license': 'http://creativecommons.org/licens...
10738562,Animalia,Chordata,Mammalia,Carnivora,Phocidae,Phoca,Phoca vitulina,332,{'license': 'http://creativecommons.org/licens...


In [20]:
group_id = df_gpd.groupby("species")["unique_id"].apply(list).reset_index()

In [21]:
group_rights = df_gpd.groupby("species")["rights"].apply(list).reset_index()

In [22]:
# ecos5 = df_gpd.groupby(['scientific_name'], as_index=False)


# %%


# ecos5 = ecos5.apply(lambda x: x)
unique_species = df_gpd.drop_duplicates(subset=["species"])
unique_species = unique_species.drop(["unique_id", "rights"], axis=1)


# %%

In [23]:
del df_gpd, rights

In [24]:
merged_groups = pd.merge(group_id, group_rights, on="species", how="left").reindex(
    columns=["species", "unique_id", "rights"]
)

In [25]:
del group_id, group_rights

In [26]:
total_merge = pd.merge(merged_groups, unique_species, on="species", how="left")

In [27]:
del merged_groups, unique_species

In [28]:
# remove duplicates values from list in unique_id column
total_merge["unique_id"] = total_merge["unique_id"].apply(set).apply(list)

In [29]:
total_merge


,species,unique_id,rights,kingdom,phylum,class,order,family,genus
0,Abietinaria abietina,"[846, 847, 639, 659]","[{'license': None, 'rightsHolder': None}, {'li...",Animalia,Cnidaria,Hydrozoa,Leptothecata,Sertulariidae,Abietinaria
1,Abietinaria filicula,"[847, 639, 659]","[{'license': None, 'rightsHolder': None}, {'li...",Animalia,Cnidaria,Hydrozoa,Leptothecata,Sertulariidae,Abietinaria
2,Abietinaria immersa,[951],"[{'license': None, 'rightsHolder': None}]",Animalia,Cnidaria,Hydrozoa,Leptothecata,Sertulariidae,Abietinaria
3,Abludomelita gladiosa,"[846, 847, 632]","[{'license': None, 'rightsHolder': None}, {'li...",Animalia,Arthropoda,Malacostraca,Amphipoda,Melitidae,Abludomelita
4,Abludomelita obtusata,"[846, 847, 639, 659]","[{'license': None, 'rightsHolder': None}, {'li...",Animalia,Arthropoda,Malacostraca,Amphipoda,Melitidae,Abludomelita
...,...,...,...,...,...,...,...,...,...
8415,Zygophylax sibogae,"[951, 1080]","[{'license': None, 'rightsHolder': None}, {'li...",Animalia,Cnidaria,Hydrozoa,Leptothecata,Zygophylacidae,Zygophylax
8416,Zygophylax tizardensis,"[951, 1080]","[{'license': None, 'rightsHolder': None}, {'li...",Animalia,Cnidaria,Hydrozoa,Leptothecata,Zygophylacidae,Zygophylax
8417,Zygothuria candelabri,[1081],"[{'license': None, 'rightsHolder': None}, {'li...",Animalia,Echinodermata,Holothuroidea,Holothuriida,Mesothuriidae,Zygothuria
8418,Zygothuria connectens,[1081],"[{'license': None, 'rightsHolder': None}]",Animalia,Echinodermata,Holothuroidea,Holothuriida,Mesothuriidae,Zygothuria


In [30]:
rights = total_merge["rights"].to_list()

In [ ]:
# tester of removing duplicate dicts from list
# rights_43 = rights[43]

# only keep unique values from rights_43 list
# rights_43 = [i for n, i in enumerate(rights_43) if i not in rights_43[n + 1:]]

In [31]:
# VERY SLOW!!! TOOK 4 HOURS ON 8500 ROWS(10 MILLION COMPRESSED)
# remove duplicate dictionaries from list of lists
for j in range(len(rights)):
    rights[j] = [i for n, i in enumerate(
        rights[j]) if i not in rights[j][n + 1:]]


In [32]:
# check some rights values. doesn't show up properly in variable explorer
rights[103]


[{'license': None, 'rightsHolder': None}]

In [33]:
# convert to series
rights = pd.Series(rights, name="rights")

total_merge["rights"] = rights


In [34]:
del rights, j

In [35]:
# rename species column to scientific_name
total_merge = total_merge.rename(columns={"species": "scientific_name"})


In [36]:
total_merge = total_merge[
    [
        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus",
        "scientific_name",
        "unique_id",
        "rights",
    ]
]

In [65]:
total_merge


[{'license': None, 'rightsHolder': None}]

In [38]:
# read cleaned df into a parquet file where user can input the file path
df_spatial_path = input("Enter the file path: ")


# write df to parquet file using pandas to_parquet
total_merge.to_parquet(df_spatial_path)
